# 0. Setup

In [1]:
# !pip install pandas==1.0.5

In [1]:
import numpy as np
import pandas as pd
import os.path
import math
import re
import random

In [2]:
root_path = os.path.dirname(os.getcwd())
ingredients_data_raw = pd.read_csv(os.path.join(root_path, 'DATA/ingredients_data.csv'))
ingredients_data = ingredients_data_raw.replace(float('nan'), '')
stir_fry_data_impractical = ingredients_data[ingredients_data['stir_fry']=='y']
stir_fry_data_all = stir_fry_data_impractical[(stir_fry_data_impractical['stir_fry_umbrella'] != 'y') & (stir_fry_data_impractical['redirect'] != 'y')]
stir_fry_data = stir_fry_data_all[stir_fry_data_all['stir_fry_yes'] == 'y']

stir_fry_data_basic = stir_fry_data[stir_fry_data['stir_fry_basic'] == 'y']
# stir_fry_data_current = stir_fry_data[stir_fry_data['2020_7_5'] == 'y']
# stir_fry_data_with_umbrella = ingredients_data[(ingredients_data['stir_fry_umbrella'] != 'y') & (ingredients_data['stir_fry'] == 'y')]
# stir_fry_data = stir_fry_data_basic
# stir_fry_data = stir_fry_data_current
# stir_fry_data = stir_fry_data_with_umbrella
stir_fry_data_is_basic = False

stir_fry_data.reset_index(drop=True, inplace=True)

In [3]:
root_path = os.path.dirname(os.getcwd())
ingredients_data_raw = pd.read_csv(os.path.join(root_path, 'DATA/ingredients_data.csv'))
ingredients_data = ingredients_data_raw.replace(float('nan'), '')

stir_fry_data = ingredients_data[(ingredients_data['stir_fry_yes'] == 'y') & (ingredients_data['stir_fry_umbrella'] != 'y') & (ingredients_data['redirect'] != 'y')]
stir_fry_data.reset_index(drop=True, inplace=True)

# 1. Matching "pairs with" terms to ingredient names
### (doesn't need to be run regularly) (except the first bit?)

In [4]:
def get_terms_from_pairs_with(pairs_with):
    if str(pairs_with) == 'nan':
        return []
    else:
        return [term.strip() for term in pairs_with.split('\n\n') if term.strip() != '']

# break entries in column that has 'pairs with' strings into lists of ingredient terms
ingredient_pairs_with_terms = stir_fry_data['pairs_with'].apply(get_terms_from_pairs_with)

In [634]:
# # create list of all terms, ignoring case and excluding duplicates
# all_terms = list(set(ingredient_pairs_with_terms.sum()))
# all_terms_lower = list(set([term.lower() for term in ingredient_pairs_with_terms.sum()]))

In [635]:
# # !pip install inflect
# import inflect
# p = inflect.engine()

In [636]:
# %%time

# salad_matches = pd.read_csv(os.path.join(root_path, 'DATA/term_name_matches_specific.csv'))
    
# def get_tokens(phrase):
#     tokens = [token.strip() for token in re.split('\(|\)|,|e\.g\.|esp\.|and|—|or|aka|see|see also|;|and\/or|\*', phrase)]
# #     print(tokens)
#     tokens = [p.singular_noun(token) or token for token in tokens if token != '']
# #     print(tokens)
#     return tokens

# def get_mark(name, term):
# #     print()
# #     print('NAME', name)
# #     print('TERM', term)
#     try:
#         salad_match = salad_matches[term][salad_matches['name'] == name].iloc[0]
# #         print('SALAD MATCH', salad_match)
#     except:
# #         print('NO SALAD MATCH')
#         salad_match = None 
#     if salad_match:
#         if str(salad_match) in ['0', 'nan']:
# #             print('BAD SALAD MATCH')
#             return ''
#         else:
# #             print('GOOD SALAD MATCH', salad_match)
# #             print('NAME', name)
# #             print('TERM', term)
# #             print()
#             return salad_match
    
#     name_tokens = [token.lower() for token in get_tokens(name)]  
#     term_tokens_mixed = get_tokens(term)
#     term_tokens = [token.lower() for token in term_tokens_mixed]
    
#     primary_name = re.split('\(|—|e\.g\.', name)[0].strip()
#     primary_name_split = primary_name.split(', ')
#     single_comma_primary_name = len(primary_name_split) == 2
    
# #     print()
#     if 'e.g.' in term:
#         if single_comma_primary_name:
#             if name_tokens[0] == term_tokens[0]:
#                 if name_tokens[1] in term_tokens: # specific name in e.g. term
# #                     print('NAME TOKENS In TERM TOKENS')
#                     term_i = term_tokens.index(name_tokens[1])
#                     if term_tokens_mixed[term_i] == term_tokens_mixed[term_i].upper():
#                         match = 'D'
#                     else:
#                         if term_tokens_mixed[0] == term_tokens_mixed[0].upper():
#                             match = 'D'
#                         else:
#                             match = 'd'
#                 else: # name matches only generic (pre e.g.) part of term
#                     if term_tokens_mixed[0] == term_tokens_mixed[0].upper():
#                         match = 'C'
#                     else:
#                         match = 'c'
#             else:
#                 match = ''
#         else:
#             if name_tokens[0] == term_tokens[0]: # name matches term before e.g.
#                 if term_tokens_mixed[0] == term_tokens_mixed[0].upper():
#                     match = 'C'
#                 else:
#                     match = 'c'
#             elif name_tokens[0] in term_tokens[1:]: # name matches term after e.g.
#                 term_i = term_tokens.index(name_tokens[0])
#                 if term_tokens_mixed[term_i] == term_tokens_mixed[term_i].upper():
#                     match = 'D'
#                 else:
#                     if term_tokens_mixed[0] == term_tokens_mixed[0].upper():
#                         match = 'D'
#                     else:
#                         match = 'd'
#             else:
#                 match = ''
#     else:
#         if single_comma_primary_name:
#             if ' '.join(primary_name_split).lower() in term.lower() or primary_name.lower() in term.lower(): # if primary_name is in term, any order
#                 if term_tokens_mixed[0] == term_tokens_mixed[0].upper():
#                     match = 'D'
#                 else:
#                     match = 'd'
#             elif name_tokens[0] == term_tokens[0]: # if first part of primary name matches first token in term
#                 if term_tokens_mixed[0] == term_tokens_mixed[0].upper():
#                     match = 'C'
#                 else:
#                     match = 'c'
#             else:
#                 match = ''
#         else:
#             if name_tokens[0] in term_tokens: # if non-comma name anywhere in non-e.g. term_tokens
#                 if term_tokens_mixed[0] == term_tokens_mixed[0].upper():
#                     match = 'D'
#                 else:
#                     match = 'd'
#             elif len(set(name_tokens).intersection(set(term_tokens))) > 0: # if there are any common tokens
#                 if term_tokens_mixed[0] == term_tokens_mixed[0].upper():
#                     match = 'C'
#                 else:
#                     match = 'c'
#             else:
#                 match = ''
                
#     if match == '':
#         n_common = len(set(name_tokens).intersection(set(term_tokens)))
#         if n_common != 0:
#             match = n_common
#     return match

In [637]:
# %%time

# # doing it this way so I can add 'print' to monitor progress
# mark_data = []
# for name in stir_fry_data['name']:
#     print(name)
#     mark_data.append([get_mark(name, term) for term in all_terms])

# term_name_marks = pd.DataFrame(mark_data, columns = all_terms)
# term_name_marks['name'] = pd.Series(stir_fry_data['name'].values.tolist())

# term_name_marks.to_csv(os.path.join(root_path, 'DATA/stir_fry_term_name_marks.csv'), index=False)

# 2. Extracting "pairs with" data

In [7]:
# term_name_matches_raw = pd.read_csv(os.path.join(root_path, 'DATA/stir_fry_term_name_matches_plus.csv'))
# term_name_matches = term_name_matches_raw.replace(['0', '1', '2', '3', '4', '5', 0, 1, 2, 3, 4, 5, float('nan')], '')

In [8]:
# %%time

# # CREATE PAIRING DATA MATRIX (names x names)
# # takes a few minutes

# pairing_data = pd.DataFrame({
#     'name': stir_fry_data['name'],
#     'pairs_with_terms': ingredient_pairs_with_terms
# })

# for name in stir_fry_data['name']:
#     pairing_data[name] = pd.Series(['']*len(stir_fry_data['name']))

# def get_pairs_with_names(row):
#     lower_category_names = []
#     lower_direct_names = []
#     upper_category_names = []
#     upper_direct_names = []
#     for term in row['pairs_with_terms']:
#         if term in term_name_matches.columns.values.tolist():
#             lower_category_names += term_name_matches[term_name_matches[term] == 'c']['name'].values.tolist()
#             lower_direct_names += term_name_matches[term_name_matches[term] == 'd']['name'].values.tolist()
#             upper_category_names += term_name_matches[term_name_matches[term] == 'C']['name'].values.tolist()
#             upper_direct_names += term_name_matches[term_name_matches[term] == 'D']['name'].values.tolist()
#         else:
#             pass
#             print(term)
    
#     for lower_category_name in lower_category_names:
#         row[lower_category_name] = 'c'
#     for lower_direct_name in lower_direct_names:
#         row[lower_direct_name] = 'd'
#     for upper_category_name in upper_category_names:
#         row[upper_category_name] = 'C'
#     for upper_direct_name in upper_direct_names:
#         row[upper_direct_name] = 'D'

#     return row

# pairing_data = pairing_data.apply(get_pairs_with_names, axis=1)
# pairing_data.replace(float('nan'), '', inplace=True)

(Native and North) American cuisines
BAKED GOODS, e.g., breads, cakes, cookies, pastries, pies, scones
butter, e.g., brown
cereals, breakfast, e.g., granola
cheese, e.g., blue, goat, ricotta
cherries, esp. dried
ice cream, e.g., butter pecan
PIES, e.g., pecan, sweet potato
purees, vegetable
waffles
wheat germ
bibimbap
cereals, hot breakfast, e.g., with fruit and nuts
“meatballs” (e.g., brown rice + onion + parsley + walnuts)
soups, e.g., tomato
sprouts, e.g., bean, pea
sweeteners, e.g., mirin, brown sugar
milk, e.g., nondairy (almond, coconut, rice)
cheese, e.g., blue, mozzarella
citrus, e.g., grapefruit, lemon, lime, orange, tangerine; juice, zest
eggs, e.g., frittatas, hard-boiled, tortillas
OILS, e.g., nut, olive, walnut
salad dressings, e.g., sherry vinaigrette
SALADS, e.g., fruit, vegetable
sauces, e.g., butter
SPANISH CUISINE
vinegar, other, e.g., balsamic, red wine, white wine
CPU times: user 27.2 s, sys: 7.79 ms, total: 27.2 s
Wall time: 27.2 s


In [9]:
# %%time

# # SYNC PAIRING DATA MATRIX (make sure [a][b] agrees with [b][a])

# for index_1, name_1 in enumerate(pairing_data['name'].values.tolist()):
#     for index_2, name_2 in enumerate(pairing_data['name'].values.tolist()):
#         value_1 = pairing_data[name_1][index_2]
#         value_2 = pairing_data[name_2][index_1]
        
#         if name_1 == name_2:
#             proper_value = ''
#         elif value_1 == 'D' or value_2 == 'D':
#             proper_value = 'D'
#         elif value_1 == 'C' or value_2 == 'C':
#             proper_value = 'C'
#         elif value_1 == 'd' or value_2 == 'd':
#             proper_value = 'd'
#         elif value_1 == 'c' or value_2 == 'c':
#             proper_value = 'c'
#         else:
#             proper_value = ''
        
#         pairing_data[name_1][index_2] = proper_value

        
#         pairing_data[name_2][index_1] = proper_value

CPU times: user 6.25 s, sys: 152 ms, total: 6.41 s
Wall time: 6.27 s


In [10]:
# %%time

# # REPRESENT PAIRING DATA AS LISTS

# def get_pairs_with_names(row):
#     lower_category_name = pairing_data[pairing_data[row['name']] == 'c']['name'].values.tolist()
#     lower_direct_name = pairing_data[pairing_data[row['name']] == 'd']['name'].values.tolist()
#     upper_category_name = pairing_data[pairing_data[row['name']] == 'C']['name'].values.tolist()
#     upper_direct_name = pairing_data[pairing_data[row['name']] == 'D']['name'].values.tolist()
    
#     row['lower_category_names'] = pairing_data[pairing_data[row['name']] == 'c']['name'].values.tolist()
#     row['lower_direct_names'] = pairing_data[pairing_data[row['name']] == 'd']['name'].values.tolist()
#     row['upper_category_names'] = pairing_data[pairing_data[row['name']] == 'C']['name'].values.tolist()
#     row['upper_direct_names'] = pairing_data[pairing_data[row['name']] == 'D']['name'].values.tolist()
#     row['lower_names'] = row['lower_category_names'] + row['lower_direct_names']
#     row['upper_names'] = row['upper_category_names'] + row['upper_direct_names']
#     row['all_names'] = row['lower_names'] + row['upper_names']
    
# #     row['lower_category_pairs_with_names'] = list(set([lower_category_name for lower_category_name in row['lower_category_names'] if lower_category_name != row['name']]))
# #     row['lower_direct_pairs_with_names'] = list(set([lower_direct_name for lower_direct_name in row['lower_direct_names'] if lower_direct_name != row['name']]))
# #     row['upper_category_pairs_with_names'] = list(set([upper_category_name for upper_category_name in row['upper_category_names'] if upper_category_name != row['name']]))
# #     row['upper_direct_pairs_with_names'] = list(set([upper_direct_name for upper_direct_name in row['upper_direct_names'] if upper_direct_name != row['name']]))
# #     row['lower_pairs_with_names'] = list(set(row['lower_category_pairs_with_names'] + row['lower_direct_pairs_with_names']))
# #     row['upper_pairs_with_names'] = list(set(row['upper_category_pairs_with_names'] + row['upper_direct_pairs_with_names']))
# #     row['all_pairs_with_names'] = list(set(row['lower_pairs_with_names'] + row['upper_pairs_with_names']))
    
#     row['lc_sorted_pairs'] = [tuple(sorted((row['name'], other_name,))) for other_name in row['lower_category_names']]
#     row['ld_sorted_pairs'] = [tuple(sorted((row['name'], other_name,))) for other_name in row['lower_direct_names']]
#     row['uc_sorted_pairs'] = [tuple(sorted((row['name'], other_name,))) for other_name in row['upper_category_names']]
#     row['ud_sorted_pairs'] = [tuple(sorted((row['name'], other_name,))) for other_name in row['upper_direct_names']]
#     row['l_sorted_pairs'] = [tuple(sorted((row['name'], other_name,))) for other_name in row['lower_names']]
#     row['u_sorted_pairs'] = [tuple(sorted((row['name'], other_name,))) for other_name in row['upper_names']]
#     row['a_sorted_pairs'] = [tuple(sorted((row['name'], other_name,))) for other_name in row['all_names']]
    
#     return row

# pairing_data = pairing_data.apply(get_pairs_with_names, axis=1)

CPU times: user 4.6 s, sys: 7.94 ms, total: 4.6 s
Wall time: 4.61 s


In [5]:
# small enough to git commit
# pairing_data.to_pickle(os.path.join(root_path, 'DATA/stir_fry_pairing_data.pickle'))
pairing_data = pd.read_pickle(os.path.join(root_path, 'DATA/stir_fry_pairing_data.pickle'))

# 3. Creating "clashes with" data \[gonna start out without this one, for stir fry\]
### (but still run through, to add it to data just in case?)

In [12]:
# name_name_clashes_blank = stir_fry_data[['name', 'protein_cheese_sub', 'stir_fry_allium', 'fruit', 'veg']].copy()
# names = stir_fry_data['name'].values.tolist()

# for i, col_name in enumerate(names):
#     name_name_clashes_blank[col_name] = pd.Series(['x']*(i+1) + ['']*(len(names)-(i+1)))

# name_name_clashes_blank.to_csv(os.path.join(root_path, 'DATA/name_name_clashes_blank.csv'), index=False)

In [13]:
# name_name_clashes_input = pd.read_csv(os.path.join(root_path, 'DATA/name_name_clashes_input.csv'))
# name_name_clashes_input = name_name_clashes_input.replace([float('nan'), 'x'], '')

In [14]:
# columns = ['name'] + name_name_clashes_input['name'].values.tolist()
# name_name_clashes_input = name_name_clashes_input[columns]

# def get_clashes_with_data(row):
#     data = pd.Series([])
#     name = row['name']
#     data['name'] = name
    
#     lower_names = name_name_clashes_input['name'][name_name_clashes_input[name] == 'y'].values.tolist()
#     upper_names = name_name_clashes_input['name'][name_name_clashes_input[name] == 'Y'].values.tolist()
#     if 'y' in lower_names:
#         print(lower_names)
    
#     for name in name_name_clashes_input['name']:
#         if row[name] == 'y':
#             lower_names.append(name)
#         elif row[name] == 'Y':
#             upper_names.append(name)
    
#     lower_names = list(set(lower_names))
#     upper_names = list(set(upper_names))
    
#     data['lower_clashes_with_names'] = lower_names
#     data['upper_clashes_with_names'] = upper_names
#     data['all_clashes_with_names'] = list(set(lower_names + upper_names)) # shouldn't be overlap here, but hey
    
#     data['lower_clashes_with_pairs'] = [tuple(sorted([name, lower_name])) for lower_name in data['lower_clashes_with_names']]
#     data['upper_clashes_with_pairs'] = [tuple(sorted([name, lower_name])) for lower_name in data['upper_clashes_with_names']]
#     data['all_clashes_with_pairs'] = [tuple(sorted([name, lower_name])) for lower_name in data['all_clashes_with_names']]
    
#     return data

# clashes_with_data = name_name_clashes_input.apply(get_clashes_with_data, axis=1)

/home/russell/flavor_tool/venv/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  """


In [15]:
# # SYNC PAIRING DATA MATRIX (make sure [a][b] agrees with [b][a])

# name_name_clashes_synced = name_name_clashes_input.copy()

# for index_1, name_1 in enumerate(name_name_clashes_input['name'].values.tolist()):
#     for index_2, name_2 in enumerate(name_name_clashes_input['name'].values.tolist()):
#         value_1 = name_name_clashes_input[name_1][index_2]
#         value_2 = name_name_clashes_input[name_2][index_1]
        
#         if name_1 == name_2:
#             proper_value = ''
#         elif value_1 == 'Y' or value_2 == 'Y':
#             proper_value = 'Y'
#         elif value_1 == 'y' or value_2 == 'y':
#             proper_value = 'y'
#         else:
#             proper_value = ''
        
#         name_name_clashes_synced[name_1][index_2] = proper_value
#         name_name_clashes_synced[name_2][index_1] = proper_value

In [16]:
# # add name_name_clashes_synced to clashes_with_data
# clashes_with_data[name_name_clashes_synced['name'].tolist()] = name_name_clashes_synced[name_name_clashes_synced['name'].tolist()].copy()

In [6]:
# clashes_with_data.to_pickle(os.path.join(root_path, 'DATA/stir_fry_clashes_with_data.pickle'))
clashes_with_data = pd.read_pickle(os.path.join(root_path, 'DATA/stir_fry_clashes_with_data.pickle'))

# 4. Adding umbrella data
- Because if 'onion' umbrella pairs with eg. olive oil, all onions should be paired with olive oil as at least c/C.
- So what I'm doing here is
    - matching umbrella-names' terms to regular ol names
    - then marking all ingredients that fall under a specific umbrella
    - then for each reg-term for each umbrella-name, I'll add a c/C connection value if existing connection is lower
- and let the records show I'm deciding that
    - if onions D pair with olive oil, but white onions c pair with olive oil, then imma overwrite c with D
    - and if onions c pair, white onions D pair I'd write D for white onions
    - (so I'm going with the bigger mark, just like when syncing)
### (also, the first part of this doesn't need to be run regularly)

In [20]:
# umbrella_data = ingredients_data[(ingredients_data['stir_fry_yes'] == 'y') & (ingredients_data['stir_fry_useful_umbrella'] == 'y') & (ingredients_data['redirect'] != 'y')]
# umbrella_data.reset_index(drop=True, inplace=True)

# stir_fry_data_with_umbrella = ingredients_data[(ingredients_data['stir_fry_yes'] == 'y') & (ingredients_data['redirect'] != 'y')]
# stir_fry_data_with_umbrella.reset_index(drop=True, inplace=True)

In [23]:
# # break entries in column that has 'pairs with' strings into lists of ingredient terms
# umbrella_ingredient_pairs_with_terms = umbrella_data['pairs_with'].apply(get_terms_from_pairs_with)

In [23]:
# # create list of all terms, ignoring case and excluding duplicates
# umbrella_all_terms = list(set(umbrella_ingredient_pairs_with_terms.sum()))
# umbrella_all_terms_lower = list(set([term.lower() for term in umbrella_ingredient_pairs_with_terms.sum()]))

In [24]:
# # !pip install inflect
# import inflect
# p = inflect.engine()

In [25]:
# salad_matches = pd.read_csv(os.path.join(root_path, 'DATA/term_name_matches_specific.csv'))
# stir_fry_matches = pd.read_csv(os.path.join(root_path, 'DATA/stir_fry_term_name_matches_plus.csv'))
    
# def get_tokens(phrase):
#     tokens = [token.strip() for token in re.split('\(|\)|,|e\.g\.|esp\.|and|—|or|aka|see|see also|;|and\/or|\*', phrase)]
#     tokens = [p.singular_noun(token) or token for token in tokens if token != '']
#     return tokens

# def get_mark(name, term):
#     try:
#         stir_fry_match = stir_fry_matches[term][stir_fry_matches['name'] == name].iloc[0]
#     except:
#         stir_fry_match = None 
#     if stir_fry_match:
#         if str(stir_fry_match) in ['0', 'nan']:
#             return ''
#         else:
#             print('FOUND stir fry match')
#             return stir_fry_match
    
#     try:
#         salad_match = salad_matches[term][salad_matches['name'] == name].iloc[0]
#     except:
#         salad_match = None 
#     if salad_match:
#         if str(salad_match) in ['0', 'nan']:
#             return ''
#         else:
#             print('FOUND salad match')
#             return salad_match
    
#     name_tokens = [token.lower() for token in get_tokens(name)]  
#     term_tokens_mixed = get_tokens(term)
#     term_tokens = [token.lower() for token in term_tokens_mixed]
    
#     primary_name = re.split('\(|—|e\.g\.', name)[0].strip()
#     primary_name_split = primary_name.split(', ')
#     single_comma_primary_name = len(primary_name_split) == 2
    
# #     print()
#     if 'e.g.' in term:
#         if single_comma_primary_name:
#             if name_tokens[0] == term_tokens[0]:
#                 if name_tokens[1] in term_tokens: # specific name in e.g. term
# #                     print('NAME TOKENS In TERM TOKENS')
#                     term_i = term_tokens.index(name_tokens[1])
#                     if term_tokens_mixed[term_i] == term_tokens_mixed[term_i].upper():
#                         match = 'D'
#                     else:
#                         if term_tokens_mixed[0] == term_tokens_mixed[0].upper():
#                             match = 'D'
#                         else:
#                             match = 'd'
#                 else: # name matches only generic (pre e.g.) part of term
#                     if term_tokens_mixed[0] == term_tokens_mixed[0].upper():
#                         match = 'C'
#                     else:
#                         match = 'c'
#             else:
#                 match = ''
#         else:
#             if name_tokens[0] == term_tokens[0]: # name matches term before e.g.
#                 if term_tokens_mixed[0] == term_tokens_mixed[0].upper():
#                     match = 'C'
#                 else:
#                     match = 'c'
#             elif name_tokens[0] in term_tokens[1:]: # name matches term after e.g.
#                 term_i = term_tokens.index(name_tokens[0])
#                 if term_tokens_mixed[term_i] == term_tokens_mixed[term_i].upper():
#                     match = 'D'
#                 else:
#                     if term_tokens_mixed[0] == term_tokens_mixed[0].upper():
#                         match = 'D'
#                     else:
#                         match = 'd'
#             else:
#                 match = ''
#     else:
#         if single_comma_primary_name:
#             if ' '.join(primary_name_split).lower() in term.lower() or primary_name.lower() in term.lower(): # if primary_name is in term, any order
#                 if term_tokens_mixed[0] == term_tokens_mixed[0].upper():
#                     match = 'D'
#                 else:
#                     match = 'd'
#             elif name_tokens[0] == term_tokens[0]: # if first part of primary name matches first token in term
#                 if term_tokens_mixed[0] == term_tokens_mixed[0].upper():
#                     match = 'C'
#                 else:
#                     match = 'c'
#             else:
#                 match = ''
#         else:
#             if name_tokens[0] in term_tokens: # if non-comma name anywhere in non-e.g. term_tokens
#                 if term_tokens_mixed[0] == term_tokens_mixed[0].upper():
#                     match = 'D'
#                 else:
#                     match = 'd'
#             elif len(set(name_tokens).intersection(set(term_tokens))) > 0: # if there are any common tokens
#                 if term_tokens_mixed[0] == term_tokens_mixed[0].upper():
#                     match = 'C'
#                 else:
#                     match = 'c'
#             else:
#                 match = ''
                
#     if match == '':
#         n_common = len(set(name_tokens).intersection(set(term_tokens)))
#         if n_common != 0:
#             match = n_common
#     return match

In [26]:
# umbrella_mark_data = []
# for name in stir_fry_data['name']:
#     print(name)
#     umbrella_mark_data.append([get_mark(name, term) for term in umbrella_all_terms])

# umbrella_term_name_marks = pd.DataFrame(umbrella_mark_data, columns = umbrella_all_terms)
# umbrella_term_name_marks['name'] = pd.Series(stir_fry_data['name'].values.tolist())

# umbrella_term_name_marks.to_csv(os.path.join(root_path, 'DATA/stir_fry_useful_umbrella_term_name_marks.csv'), index=False)

In [18]:
# umbrella_term_name_matches_raw = pd.read_csv(os.path.join(root_path, 'DATA/stir_fry_umbrella_term_name_matches.csv'))
# umbrella_term_name_matches = umbrella_term_name_matches_raw.replace(['0', '1', '2', '3', '4', '5', 0, 1, 2, 3, 4, 5, float('nan')], '')

In [24]:
# %%time
# # CREATE PAIRING DATA MATRIX (names x names)

# umbrella_pairing_data = pd.DataFrame({
#     'name': umbrella_data['name'],
#     'pairs_with_terms': umbrella_ingredient_pairs_with_terms
# })

# for name in stir_fry_data['name']:
#     umbrella_pairing_data[name] = pd.Series(['']*len(umbrella_data))

# def get_pairs_with_names(row):
#     lower_category_names = []
#     lower_direct_names = []
#     upper_category_names = []
#     upper_direct_names = []
#     for term in row['pairs_with_terms']:
#         if term in umbrella_term_name_matches.columns.values.tolist():
#             lower_category_names += umbrella_term_name_matches[umbrella_term_name_matches[term] == 'c']['name'].values.tolist()
#             lower_direct_names += umbrella_term_name_matches[umbrella_term_name_matches[term] == 'd']['name'].values.tolist()
#             upper_category_names += umbrella_term_name_matches[umbrella_term_name_matches[term] == 'C']['name'].values.tolist()
#             upper_direct_names += umbrella_term_name_matches[umbrella_term_name_matches[term] == 'D']['name'].values.tolist()
#         else:
#             pass
#             print(term)
    
#     for lower_category_name in lower_category_names:
#         row[lower_category_name] = 'c'
#     for lower_direct_name in lower_direct_names:
#         row[lower_direct_name] = 'd'
#     for upper_category_name in upper_category_names:
#         row[upper_category_name] = 'C'
#     for upper_direct_name in upper_direct_names:
#         row[upper_direct_name] = 'D'

#     return row

# umbrella_pairing_data = umbrella_pairing_data.apply(get_pairs_with_names, axis=1)
# umbrella_pairing_data.replace(float('nan'), '', inplace=True)

CPU times: user 2.69 s, sys: 0 ns, total: 2.69 s
Wall time: 2.68 s


In [29]:
# # Creating template to match ingredients to their umbrellas
# umbrella_covered_blanks = pd.DataFrame({
#     umbrella_name: ['']*len(stir_fry_data)
# for umbrella_name in umbrella_data['name']})
# umbrella_covered_blanks['name'] = stir_fry_data['name']
# umbrella_covered_blanks.to_csv(os.path.join(root_path, 'DATA/stir_fry_umbrella_covered_blanks.csv'), index=False)

In [25]:
# umbrella_covered_matches_raw = pd.read_csv(os.path.join(root_path, 'DATA/stir_fry_umbrella_covered_matches.csv'))
# umbrella_covered_matches = umbrella_covered_matches_raw.replace([float('nan')], '')
# def get_covered_list(umbrella_name):
#     return umbrella_covered_matches[umbrella_covered_matches[umbrella_name] == 'y']['name'].tolist()

In [26]:
# %%time
# # 40s

# pairing_data_with_umbrella_matches = pairing_data.copy()

# for umbrella_name in umbrella_data['name']: # eg. onions
#     for covered_name in get_covered_list(umbrella_name): # eg. white onion
# #         print(umbrella_name, covered_name)
#         for stir_fry_name in stir_fry_data['name']: # eg. olive oil
#             umbrella_match_value = umbrella_pairing_data[umbrella_pairing_data['name'] == umbrella_name][stir_fry_name].iloc[0] # eg. D (because onions match with OLIVE OIL)
# #             print(umbrella_match_value, umbrella_name, stir_fry_name)
#             stir_fry_match_value = pairing_data[pairing_data['name'] == covered_name][stir_fry_name].iloc[0] # eg. 'c' (because white onions match with oil)
# #             print(umbrella_match_value, stir_fry_match_value)
        
#             if umbrella_match_value == 'D' or stir_fry_match_value == 'D':
#                 proper_match_value = 'D'
#             elif umbrella_match_value == 'C' or stir_fry_match_value == 'C':
#                 proper_match_value = 'C'
#             elif umbrella_match_value == 'd' or stir_fry_match_value == 'd':
#                 proper_match_value = 'd'
#             elif umbrella_match_value == 'c' or stir_fry_match_value == 'c':
#                 proper_match_value = 'c'
#             else:
#                 proper_match_value = ''
                
#             pairing_data_with_umbrella_matches.loc[pairing_data_with_umbrella_matches['name'] == covered_name, stir_fry_name] = proper_match_value
#             pairing_data_with_umbrella_matches.loc[pairing_data_with_umbrella_matches['name'] == stir_fry_name, covered_name] = proper_match_value

CPU times: user 38.8 s, sys: 44 ms, total: 38.9 s
Wall time: 38.9 s


In [27]:
# %%time

# # REPRESENT PAIRING DATA AS LISTS

# def get_pairs_with_names(row):
#     lower_category_name = pairing_data_with_umbrella_matches[pairing_data_with_umbrella_matches[row['name']] == 'c']['name'].values.tolist()
#     lower_direct_name = pairing_data_with_umbrella_matches[pairing_data_with_umbrella_matches[row['name']] == 'd']['name'].values.tolist()
#     upper_category_name = pairing_data_with_umbrella_matches[pairing_data_with_umbrella_matches[row['name']] == 'C']['name'].values.tolist()
#     upper_direct_name = pairing_data_with_umbrella_matches[pairing_data_with_umbrella_matches[row['name']] == 'D']['name'].values.tolist()
    
#     row['lower_category_names'] = pairing_data_with_umbrella_matches[pairing_data_with_umbrella_matches[row['name']] == 'c']['name'].values.tolist()
#     row['lower_direct_names'] = pairing_data_with_umbrella_matches[pairing_data_with_umbrella_matches[row['name']] == 'd']['name'].values.tolist()
#     row['upper_category_names'] = pairing_data_with_umbrella_matches[pairing_data_with_umbrella_matches[row['name']] == 'C']['name'].values.tolist()
#     row['upper_direct_names'] = pairing_data_with_umbrella_matches[pairing_data_with_umbrella_matches[row['name']] == 'D']['name'].values.tolist()
#     row['lower_names'] = row['lower_category_names'] + row['lower_direct_names']
#     row['upper_names'] = row['upper_category_names'] + row['upper_direct_names']
#     row['all_names'] = row['lower_names'] + row['upper_names']
    
# #     row['lower_category_pairs_with_names'] = list(set([lower_category_name for lower_category_name in row['lower_category_names'] if lower_category_name != row['name']]))
# #     row['lower_direct_pairs_with_names'] = list(set([lower_direct_name for lower_direct_name in row['lower_direct_names'] if lower_direct_name != row['name']]))
# #     row['upper_category_pairs_with_names'] = list(set([upper_category_name for upper_category_name in row['upper_category_names'] if upper_category_name != row['name']]))
# #     row['upper_direct_pairs_with_names'] = list(set([upper_direct_name for upper_direct_name in row['upper_direct_names'] if upper_direct_name != row['name']]))
# #     row['lower_pairs_with_names'] = list(set(row['lower_category_pairs_with_names'] + row['lower_direct_pairs_with_names']))
# #     row['upper_pairs_with_names'] = list(set(row['upper_category_pairs_with_names'] + row['upper_direct_pairs_with_names']))
# #     row['all_pairs_with_names'] = list(set(row['lower_pairs_with_names'] + row['upper_pairs_with_names']))
    
#     row['lc_sorted_pairs'] = [tuple(sorted((row['name'], other_name,))) for other_name in row['lower_category_names']]
#     row['ld_sorted_pairs'] = [tuple(sorted((row['name'], other_name,))) for other_name in row['lower_direct_names']]
#     row['uc_sorted_pairs'] = [tuple(sorted((row['name'], other_name,))) for other_name in row['upper_category_names']]
#     row['ud_sorted_pairs'] = [tuple(sorted((row['name'], other_name,))) for other_name in row['upper_direct_names']]
#     row['l_sorted_pairs'] = [tuple(sorted((row['name'], other_name,))) for other_name in row['lower_names']]
#     row['u_sorted_pairs'] = [tuple(sorted((row['name'], other_name,))) for other_name in row['upper_names']]
#     row['a_sorted_pairs'] = [tuple(sorted((row['name'], other_name,))) for other_name in row['all_names']]
    
#     return row

# pairing_data_with_umbrella_matches = pairing_data_with_umbrella_matches.apply(get_pairs_with_names, axis=1)

CPU times: user 2.07 s, sys: 0 ns, total: 2.07 s
Wall time: 2.07 s


In [7]:
# pairing_data_with_umbrella_matches.to_pickle(os.path.join(root_path, 'DATA/stir_fry_pairing_data_with_umbrella_matches.pickle'))
pairing_data_with_umbrella_matches = pd.read_pickle(os.path.join(root_path, 'DATA/stir_fry_pairing_data_with_umbrella_matches.pickle'))

# 5. Creating connection data \[without clashes data, for now\]

In [31]:
# %%time
# # 23s

# # CREATING CONNECTION DATA AND ADDING IT TO stir_fry DATA (& getting stir_fry_data in order)
# # takes a minute
# stir_fry_names = stir_fry_data['name'].values.tolist()
# for col_name in stir_fry_names:
#     col_values = []
#     for row_name in stir_fry_names:
#         pairing_value = pairing_data_with_umbrella_matches[col_name][pairing_data_with_umbrella_matches['name'] == row_name].iloc[0]
# #         clashing_value = clashes_with_data[col_name][clashes_with_data['name'] == row_name].iloc[0]

# #         if clashing_value == 'Y':
# #             col_values.append('N')
# #         elif clashing_value == 'y':
# #             col_values.append('n')
# #         elif pairing_value == 'D':
# #             col_values.append('D')
# #         elif pairing_value == 'C':
# #             col_values.append('C')
# #         elif pairing_value == 'd':
# #             col_values.append('d')
# #         elif pairing_value == 'c':
# #             col_values.append('c')
# #         else:
# #             col_values.append('')
#         col_values.append(pairing_value)

#     stir_fry_data[col_name] = pd.Series(col_values)
    
# # just in case
# stir_fry_data.sort_values('name', inplace=True)
# stir_fry_data.reset_index(drop=True, inplace=True)

/home/russell/flavor_tool/venv/lib/python3.6/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


CPU times: user 27.1 s, sys: 11.7 ms, total: 27.1 s
Wall time: 27.1 s


/home/russell/flavor_tool/venv/lib/python3.6/site-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [8]:
# stir_fry_data.to_pickle(os.path.join(root_path, 'DATA/stir_fry_data_with_names_and_umbrella_matches.pickle'))
stir_fry_data = pd.read_pickle(os.path.join(root_path, 'DATA/stir_fry_data_with_names_and_umbrella_matches.pickle'))

# 6. Creating flavor tool data

In [33]:
# stir_fry_flavor_data = stir_fry_data[['staple', 'not_vegan', 'gluten', 'flavoring_sweet', 'flavoring_fresh', 'flavoring_wet', 'flavoring_concentrate', 'flavoring', 'flavoring_dry', 'protein', 'protein_cheese_sub', 'protein_milk_sub', 'protein_meat_sub', 'protein_bean', 'veg', 'veg_leafy', 'grain', 'grain_flour', 'fat_oil', 'oil', 'fat', 'fruit', 'fruit_berry', 'stir_fry', 'stir_fry_basic', 'stir_fry_yes', 'stir_fry_early', 'stir_fry_mid', 'stir_fry_late', 'stir_fry_garnish', 'stir_fry_umbrella', 'protein_nut_seed', 'protein_nut', 'protein_seed', 'redirect', 'salty', 'sour', 'spicy', 'bitter', 'savory', 'sweet','name', 'flavor', 'volume', 'pairs_with', 'phonetic', 'techniques', 'dishes', 'tip', 'possible_substitutes', 'flavor_affinities', 'nutritional_profile', 'season', 'botanical_relatives', 'protein_content', 'what_they_are', 'brands', 'vegan_substitutes', 'vegan_brands', 'stir_fry_protein', 'stir_fry_protein_nut_seed', 'stir_fry_protein_nut', 'stir_fry_protein_seed', 'stir_fry_flavoring', 'stir_fry_oil', 'stir_fry_fat', 'stir_fry_fat_oil', 'stir_fry_fruit', 'stir_fry_allium', 'stir_fry_protein_bean', 'stir_fry_veg', 'stir_fry_grain', 'stir_fry_salt', 'stir_fry_pepper', 'stir_fry_vinegar', 'strong', 'stir_fry_mushroom', 'stir_fry_main', 'stir_fry_useful_umbrella'] + stir_fry_data['name'].tolist()].copy()
# stir_fry_flavor_data['id'] = range(len(stir_fry_flavor_data))

# selected_pairing_data = pairing_data_with_umbrella_matches[['name', 'pairs_with_terms', 'lower_category_names', 'lower_direct_names', 'upper_category_names', 'upper_direct_names', 'lower_names', 'upper_names', 'all_names', 'lc_sorted_pairs', 'ld_sorted_pairs', 'uc_sorted_pairs', 'ud_sorted_pairs', 'l_sorted_pairs', 'u_sorted_pairs', 'a_sorted_pairs']]
# stir_fry_flavor_data = stir_fry_flavor_data.merge(selected_pairing_data, how='inner', on='name')

# # selected_clashes_with_data = clashes_with_data[['name', 'lower_clashes_with_names', 'upper_clashes_with_names', 'all_clashes_with_names', 'lower_clashes_with_pairs', 'upper_clashes_with_pairs', 'all_clashes_with_pairs']]
# # stir_fry_flavor_data = stir_fry_flavor_data.merge(selected_clashes_with_data, how='inner', on='name')

# stir_fry_flavor_data.replace(float('nan'), '', inplace=True)
# stir_fry_flavor_data.set_index('name', drop=False, inplace=True)
# stir_fry_flavor_data.rename_axis('name_index', inplace=True)

In [34]:
# %%time
# # 13s

# # ADD STRENGTH VALUES
# ref_flavor_data = stir_fry_flavor_data.copy()
# for i_1, row_1 in ref_flavor_data.iterrows():
#     for i_2, row_2 in ref_flavor_data.iterrows():
#         pairs_with_value = ref_flavor_data[row_1['name']][row_2['name']] or '_'
#         if row_1['strong'] in ['y', 'Y'] and row_2['strong'] in ['y', 'Y']:
#             strength_value = 'S'
#         elif row_1['strong'] in ['y', 'Y'] or row_2['strong'] in ['y', 'Y']:
#             strength_value = 's'
#         else:
#             strength_value = '_'
            
#         stir_fry_flavor_data[row_1['name']][row_2['name']] = pairs_with_value + strength_value

/home/russell/flavor_tool/venv/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


CPU times: user 13.1 s, sys: 40 ms, total: 13.1 s
Wall time: 13.1 s


In [9]:
# stir_fry_flavor_data.to_pickle(os.path.join(root_path, 'DATA/stir_fry_flavor_data_with_umbrella.pickle'))
# stir_fry_flavor_data.to_pickle(os.path.join(root_path, '../data/stir_fry_flavor_data_with_umbrella.pickle'))
stir_fry_flavor_data = pd.read_pickle(os.path.join(root_path, 'DATA/stir_fry_flavor_data_with_umbrella.pickle'))

# 7. stir_fry recipe generators

## 7.1. Regular generator

In [10]:
# !pip install networkx==2.4
# !pip install pyvis

import networkx as nx
from pyvis import network as net
import random

In [15]:
n_salts = 1
n_fat_oils = 1
n_other_flavorings_min = 1
n_other_flavorings_max = 3
n_foodstuffs_min = 3
n_foodstuffs_max = 7

# n_locked = random.randrage(0, 4)
n_locked = 3
locked_names = stir_fry_flavor_data.sample(n_locked)['name'].tolist()

n_present = len(stir_fry_flavor_data)
present_names = stir_fry_flavor_data.sample(n_present)['name'].tolist()

stir_fry_data = stir_fry_flavor_data[stir_fry_flavor_data['name'].isin(present_names)].copy()

locked = stir_fry_data[stir_fry_data['name'].isin(locked_names)]
locked_fat_oils = locked[locked['stir_fry_fat_oil'] == 'y']
locked_salts = locked[locked['stir_fry_salt'] == 'y']
locked_other_flavorings = locked[(locked['stir_fry_flavoring'] == 'y') & (locked['stir_fry_salt'] != 'y')]
locked_foodstuffs = locked[(locked['stir_fry_fat_oil'] != 'y') & (locked['stir_fry_salt'] != 'y') & (locked['stir_fry_flavoring'] != 'y')]

the_rest = stir_fry_data[~stir_fry_data['name'].isin(locked['name'])]
the_rest_fat_oils = the_rest[the_rest['stir_fry_fat_oil'] == 'y']
the_rest_salts = the_rest[the_rest['stir_fry_salt'] == 'y']
the_rest_other_flavorings = the_rest[(the_rest['stir_fry_flavoring'] == 'y') & (the_rest['stir_fry_salt'] != 'y')]
the_rest_foodstuffs = the_rest[(the_rest['stir_fry_fat_oil'] != 'y') & (the_rest['stir_fry_salt'] != 'y') & (the_rest['stir_fry_flavoring'] != 'y')]

n_additional_salts_needed = max(n_salts - len(locked_salts), 0)
n_additional_salts_actual = min(n_additional_salts_needed, len(the_rest_salts))
n_total_salts_actual = n_additional_salts_actual + len(locked_salts)

n_additional_fat_oils_needed = max(n_fat_oils - len(locked_fat_oils), 0)
n_additional_fat_oils_actual = min(n_additional_fat_oils_needed, len(the_rest_fat_oils))
n_total_fat_oils_actual = n_additional_fat_oils_actual + len(locked_fat_oils)

n_additional_other_flavorings_needed_min = max(n_other_flavorings_min - len(locked_other_flavorings), 0)
n_additional_other_flavorings_actual_min = min(n_additional_other_flavorings_needed_min, len(the_rest_other_flavorings))
n_total_other_flavorings_actual_min = n_additional_other_flavorings_actual_min + len(locked_other_flavorings)

n_additional_other_flavorings_needed_max = max(n_other_flavorings_max - len(locked_other_flavorings), 0) # yikes, I had this as min..
n_additional_other_flavorings_actual_max = min(n_additional_other_flavorings_needed_max, len(the_rest_other_flavorings))
n_total_other_flavorings_actual_max = n_additional_other_flavorings_actual_max + len(locked_other_flavorings)

n_additional_foodstuffs_needed_min = max(n_foodstuffs_min - len(locked_foodstuffs), 0)
n_additional_foodstuffs_actual_min = min(n_additional_foodstuffs_needed_min, len(the_rest_foodstuffs))
n_total_foodstuffs_actual_min = n_additional_foodstuffs_actual_min + len(locked_foodstuffs)

n_additional_foodstuffs_needed_max = max(n_foodstuffs_max - len(locked_foodstuffs), 0)
n_additional_foodstuffs_actual_max = min(n_additional_foodstuffs_needed_max, len(the_rest_foodstuffs))
n_total_foodstuffs_actual_max = n_additional_foodstuffs_actual_max + len(locked_foodstuffs)


In [18]:
n_iterations = 300
top_score = None
for iteration in range(n_iterations):
    n_additional_other_flavorings_actual = random.randrange(n_additional_other_flavorings_actual_min, n_additional_other_flavorings_actual_max+1)
    n_additional_foodstuffs_actual = random.randrange(n_additional_foodstuffs_actual_min, n_additional_foodstuffs_actual_max+1)

    selected_ingredients = locked
    if n_additional_salts_actual > 0:
        selected_ingredients = selected_ingredients.append(the_rest_salts.sample(n_additional_salts_actual))
    if n_additional_fat_oils_actual > 0:
        selected_ingredients = selected_ingredients.append(the_rest_fat_oils.sample(n_additional_fat_oils_actual))
    if n_additional_other_flavorings_actual > 0:
        selected_ingredients = selected_ingredients.append(the_rest_other_flavorings.sample(n_additional_other_flavorings_actual))
    if n_additional_foodstuffs_actual > 0:
        selected_ingredients = selected_ingredients.append(the_rest_foodstuffs.sample(n_additional_foodstuffs_actual))

    selected_names = selected_ingredients['name'].values.tolist()

    selected_g = nx.Graph()
    selected_g.add_nodes_from(selected_names)

    for i_1, name_1 in enumerate(selected_names[:-1]):
        for i_2, name_2 in enumerate(selected_names[i_1+1:], i_1+1):
            connection = selected_ingredients[name_1][name_2]

            # Weights super guess-y
            if connection[0] == 'c':
                selected_g.add_edge(name_1, name_2, length=1, weight=.3) # prev .8
            elif connection[0] == 'd':
                # pairs_with_demerit = .5 # prev .6666
                selected_g.add_edge(name_1, name_2, length=.7, weight=.6) # prev .8
            elif connection[0] == 'C':
                # pairs_with_demerit = .4 # prev .5333
                selected_g.add_edge(name_1, name_2, length=.6, weight=.7) # prev .8
            elif connection[0] == 'D':
                # pairs_with_demerit = .3 # prev. .4
                selected_g.add_edge(name_1, name_2, length=.3, weight=1) # prev .8

    # Try again, friend
    if not nx.is_connected(selected_g):
#         print(str(iteration)+': NOT CONNECTED; SKIPPING TO NEXT ITERATION')
        continue

    score = 0

    # CONNECTED PAIRING BONUS ==============================================
    # I want this to be VERY important. I feel this holds a lot of the strength of recipe,
    # and also encompasses strength-ness and locked-ness
    # ranges from roughly (0 to 1) * 3, tho could be a lil over or under that range
    average_shortest_path_length = nx.average_shortest_path_length(selected_g, weight='length')
    pairing_score = 1 / average_shortest_path_length * 1.4 - 1 # good enough (for small, large pools)
    # print('PAIRING SCORE', pairing_score)
    score += pairing_score * 5

    # Used for both strength and locked bonus:
    node_degrees = list(selected_g.degree(weight='weight'))
    average_degree = sum([node_degree[1] for node_degree in node_degrees]) / len(node_degrees)

    # STRENGTH BONUS =======================================================
    strength_above_average = 0
    for node_degree in node_degrees:
        if selected_ingredients['strong'][node_degree[0]] == 'Y':
            strength_above_average += (node_degree[1] - average_degree)
        elif selected_ingredients['strong'][node_degree[0]] == 'y':
            strength_above_average += (node_degree[1] - average_degree) * .5
    strength_score = strength_above_average * .2 + .5 # reasonable for small and full pools
    # print('STRENGTH SCORE', strength_score)
    score += strength_score

    # LOCKED BONUS =========================================================
    locked_above_average = 0
    for node_degree in node_degrees:
        if node_degree[0] in locked_names:
            locked_above_average += node_degree[1] - average_degree
    locked_score = locked_above_average * .2 + .5 # close enough (has to cover few locked, lotta locked, small pool, big pool - yeesh.)
    # print('LOCKED SCORE', locked_score)
    score += locked_score * 2

# FLAVOR BALANCE BONUS =============================================================================================
# ranges from roughly (0 to 1) * 1 (could be a lil over/under)
    n_sweet_lower = (selected_ingredients['sweet'] == 'y').sum()
    n_sweet_upper = (selected_ingredients['sweet'] == 'Y').sum()
    n_salty_lower = (selected_ingredients['salty'] == 'y').sum()
    n_salty_upper = (selected_ingredients['salty'] == 'Y').sum()
    n_sour_lower = (selected_ingredients['sour'] == 'y').sum()
    n_sour_upper = (selected_ingredients['sour'] == 'Y').sum()
    n_savory_lower = (selected_ingredients['savory'] == 'y').sum()
    n_savory_upper = (selected_ingredients['savory'] == 'Y').sum()
    n_bitter_lower = (selected_ingredients['bitter'] == 'y').sum()
    n_bitter_upper = (selected_ingredients['bitter'] == 'Y').sum()
    n_spicy_lower = (selected_ingredients['spicy'] == 'y').sum()
    n_spicy_upper = (selected_ingredients['spicy'] == 'Y').sum()

    sweet_score = min(n_sweet_lower/2 + n_sweet_upper, 1)
    salty_score = min(n_salty_lower/2 + n_salty_upper, 1)
    sour_score = min(n_sour_lower/2 + n_sour_upper, 1)
    savory_score = min(n_savory_lower/2 + n_savory_upper, 1)
    bitter_score = min(n_bitter_lower/2 + n_bitter_upper, 1)
    spicy_score = min(n_spicy_lower/2 + n_spicy_upper, 1)

    flavor_score = 0
    flavor_score += sweet_score*3 # rly want something sweet in there
    flavor_score += salty_score*.5 # can always use salt
    flavor_score += sour_score*2 # like me some sour
    flavor_score += savory_score*3 # LOVE me some savory
    flavor_score += bitter_score # idk
    flavor_score += spicy_score*2 # can be nice
    flavor_score = flavor_score * .15 - .75

    # print('FLAVOR BALANCE SCORE', flavor_balance_score)
    score += flavor_score

    # FOOD GROUPS BONUS ==========================================================================================
    if 'y' in selected_ingredients['stir_fry_protein'].values:
        protein_score = .5
    else:
        protein_score = 0

    if 'y' in selected_ingredients['stir_fry_fruit'].values:
        fruit_score = .5
    else:
        fruit_score = 0

    food_group_score = protein_score + fruit_score
    # print('PROTEIN FRUIT', protein_score, fruit_score)
    score += food_group_score


    if top_score == None or score > top_score:
        top_selected_ingredients = selected_ingredients
        top_pairing_score = pairing_score
        top_strength_score = strength_score
        top_locked_score = locked_score
        top_flavor_score = flavor_score
        top_food_group_score = food_group_score
        top_score = score

regular_net = net.Network(notebook=True)

top_selected_names = top_selected_ingredients['name'].tolist()
nodes = top_selected_names

def get_color(row):
#     return 'grey'
    if row['stir_fry_veg'] == 'y':
        return 'green'
    elif row['stir_fry_fruit'] == 'y':
        return 'orange'
    elif row['stir_fry_protein'] == 'y':
        return 'brown'
    elif row['stir_fry_grain'] == 'y':
        return 'tan'
    else:
        return 'lightgrey'
nodes_color = top_selected_ingredients.apply(get_color, axis=1).tolist()

regular_net.add_nodes(
    nodes=nodes,
    color=nodes_color
)

for i_1, name_1 in enumerate(top_selected_names[:-1]):
    for i_2, name_2 in enumerate(top_selected_names[i_1+1:], i_1+1):
        connection = top_selected_ingredients[name_1][name_2]
#         if connection[0] == 'c':
#             clique_net.add_edge(name_1, name_2, physics=False, color='lightgrey')
        if connection[0] == 'd':
            clique_net.add_edge(name_1, name_2, physics=True, color='whitesmoke')
        elif connection[0] == 'C':
            clique_net.add_edge(name_1, name_2, physics=True, color='lightgrey')
        elif connection[0] == 'D':
            clique_net.add_edge(name_1, name_2, physics=True, color='black')

regular_net.show('regular_net.html')

## 7.2. Black magic

### 7.2.1. Preparing subgraph data
(stuff that can be pre-calculated, pickled)

In [19]:
import networkx as nx
# import community
from networkx.algorithms.clique import enumerate_all_cliques, find_cliques, cliques_containing_node

In [20]:
selected_ingredients = stir_fry_flavor_data.copy()#.sample(200)
selected_names = selected_ingredients['name'].tolist()

lower_category_pairs = []
lower_direct_pairs = []
upper_category_pairs = []
upper_direct_pairs = []

for i_1, name_1 in enumerate(selected_names[:-1]):
    for i_2, name_2 in enumerate(selected_names[i_1+1:], i_1+1):
        connection = selected_ingredients[name_1][name_2] # this is what is finicky
        if connection[0] == 'c':
            lower_category_pairs.append((name_1, name_2,))
        elif connection[0] == 'd':
            lower_direct_pairs.append((name_1, name_2,))
        elif connection[0] == 'C':
            upper_category_pairs.append((name_1, name_2,))
        elif connection[0] == 'D':
            upper_direct_pairs.append((name_1, name_2,))
lower_pairs = lower_category_pairs + lower_direct_pairs
upper_pairs = upper_category_pairs + upper_direct_pairs
all_pairs = lower_pairs + upper_pairs

In [21]:
# %%time
# G = nx.Graph()
# G.add_nodes_from(selected_names)
# G.add_edges_from(upper_category_pairs)
# G.add_edges_from(upper_direct_pairs)

# cliques_gen = find_cliques(G)
# cliques_all = list(cliques_gen)
# cliques = [clique for clique in cliques_all if len(clique) > 1]
# clique_sets = [set(clique) for clique in cliques]

# data = {
#     s_name: ['y' if s_name in c_set else '' for c_set in clique_sets]
# for s_name in selected_names}
# clique_upper = pd.DataFrame(data)
# clique_upper['type'] = ['clique_upper'] * len(clique_upper)
# clique_upper['list'] = cliques
# clique_upper['set'] = clique_sets
# clique_upper['length'] = clique_upper['list'].apply(len)

# clique_upper.to_pickle(os.path.join(root_path, 'DATA/stir_fry_clique_upper.pickle'))
clique_upper = pd.read_pickle(os.path.join(root_path, 'DATA/stir_fry_clique_upper.pickle'))

In [22]:
%%time

salt_set = set(stir_fry_flavor_data[stir_fry_flavor_data['stir_fry_salt'] == 'y']['name'])
fat_oil_set = set(stir_fry_flavor_data[stir_fry_flavor_data['stir_fry_fat_oil'] == 'y']['name'])
other_flavoring_set = set(stir_fry_flavor_data[stir_fry_flavor_data['stir_fry_flavoring'] == 'y']['name']) - salt_set
foodstuff_set = set(stir_fry_flavor_data['name']) - salt_set - fat_oil_set - other_flavoring_set
mushroom_set = set(stir_fry_flavor_data[stir_fry_flavor_data['stir_fry_mushroom'] == 'y']['name'])
bean_set = set(stir_fry_flavor_data[stir_fry_flavor_data['stir_fry_protein_bean'] == 'y'])
grain_set = set(stir_fry_flavor_data[stir_fry_flavor_data['stir_fry_grain'] == 'y'])

clique_upper['salts'] = [list(salt_set.intersection(clique_set)) for clique_set in list(clique_upper['set'])]
clique_upper['fat_oils'] = [list(fat_oil_set.intersection(clique_set)) for clique_set in list(clique_upper['set'])]
clique_upper['other_flavorings'] = [list(other_flavoring_set.intersection(clique_set)) for clique_set in list(clique_upper['set'])]
clique_upper['foodstuffs'] = [list(foodstuff_set.intersection(clique_set)) for clique_set in list(clique_upper['set'])]
clique_upper['mushrooms'] = [list(mushroom_set.intersection(clique_set)) for clique_set in list(clique_upper['set'])]
clique_upper['beans'] = [list(bean_set.intersection(clique_set)) for clique_set in list(clique_upper['set'])]
clique_upper['grains'] = [list(grain_set.intersection(clique_set)) for clique_set in list(clique_upper['set'])]

clique_upper['salts_set'] = [set(salt_set.intersection(clique_set)) for clique_set in list(clique_upper['set'])]
clique_upper['fat_oils_set'] = [set(fat_oil_set.intersection(clique_set)) for clique_set in list(clique_upper['set'])]
clique_upper['other_flavorings_set'] = [set(other_flavoring_set.intersection(clique_set)) for clique_set in list(clique_upper['set'])]
clique_upper['foodstuffs_set'] = [set(foodstuff_set.intersection(clique_set)) for clique_set in list(clique_upper['set'])]
clique_upper['mushrooms_set'] = [set(mushroom_set.intersection(clique_set)) for clique_set in list(clique_upper['set'])]
clique_upper['beans_set'] = [set(bean_set.intersection(clique_set)) for clique_set in list(clique_upper['set'])]
clique_upper['grains_set'] = [set(grain_set.intersection(clique_set)) for clique_set in list(clique_upper['set'])]

clique_upper['n_salts'] = [len(salt_set.intersection(clique_set)) for clique_set in list(clique_upper['set'])]
clique_upper['n_fat_oils'] = [len(fat_oil_set.intersection(clique_set)) for clique_set in list(clique_upper['set'])]
clique_upper['n_other_flavorings'] = [len(other_flavoring_set.intersection(clique_set)) for clique_set in list(clique_upper['set'])]
clique_upper['n_foodstuffs'] = [len(foodstuff_set.intersection(clique_set)) for clique_set in list(clique_upper['set'])]
clique_upper['n_mushrooms'] = [len(mushroom_set.intersection(clique_set)) for clique_set in list(clique_upper['set'])]


CPU times: user 279 ms, sys: 12 ms, total: 291 ms
Wall time: 290 ms


In [23]:
# THESE ARE THE ACTUAL CONTROLS FOR THE GENERATOR (I think? it's been a bit.)
n_salts = 1
n_fat_oils = 1
n_other_flavorings_min = 1
n_other_flavorings_max = 3
n_foodstuffs_min = 3
n_foodstuffs_max = 7
mushrooms_cap = 1 # possible to have more than this, if there are locked mushrooms (I don't update sets after locked/gen is established)
n_beans_max = 1
n_grains_max = 1

In [24]:
# Important, don't delete.

reasonable_clique_upper = clique_upper.copy()

In [25]:
%%time
def get_reasonable_data(row):
    # done first, since it messes w foodstuffs
    reasonable_n_mushrooms = min(row['n_mushrooms'], mushrooms_cap)
    mushrooms_to_remove_set = set(random.sample(row['mushrooms'], row['n_mushrooms'] - reasonable_n_mushrooms))
    reasonable_mushrooms_set = row['mushrooms_set'] - mushrooms_to_remove_set
    reasonable_mushrooms = list(reasonable_mushrooms_set)
    deshroomed_foodstuffs_set = row['foodstuffs_set'] - mushrooms_to_remove_set
    deshroomed_foodstuffs = list(deshroomed_foodstuffs_set)
    deshroomed_n_foodstuffs = len(deshroomed_foodstuffs)

    deshroomed_set = row['set'].copy()
    deshroomed_set -= mushrooms_to_remove_set
    deshroomed_list = list(deshroomed_set)
    deshroomed_length = len(deshroomed_list)
    
    reasonable_n_salts = min(row['n_salts'], n_salts)
    reasonable_n_fat_oils = min(row['n_fat_oils'], n_fat_oils)
    reasonable_n_other_flavorings = min(row['n_other_flavorings'], n_other_flavorings_max)
    reasonable_n_foodstuffs = min(deshroomed_n_foodstuffs, n_foodstuffs_max)

    salts_to_remove_set = set(random.sample(row['salts'], row['n_salts'] - reasonable_n_salts))
    fat_oils_to_remove_set = set(random.sample(row['fat_oils'], row['n_fat_oils'] - reasonable_n_fat_oils))
    other_flavorings_to_remove_set = set(random.sample(row['other_flavorings'], row['n_other_flavorings'] - reasonable_n_other_flavorings))
    foodstuffs_to_remove_set = set(random.sample(deshroomed_foodstuffs, deshroomed_n_foodstuffs - reasonable_n_foodstuffs))
    
    reasonable_salts_set = row['salts_set'] - salts_to_remove_set
    reasonable_fat_oils_set = row['fat_oils_set'] - fat_oils_to_remove_set
    reasonable_other_flavorings_set = row['other_flavorings_set'] - other_flavorings_to_remove_set
    reasonable_foodstuffs_set = deshroomed_foodstuffs_set - foodstuffs_to_remove_set
    
    reasonable_salts = list(reasonable_salts_set)
    reasonable_fat_oils = list(reasonable_fat_oils_set)
    reasonable_other_flavorings = list(reasonable_other_flavorings_set)
    reasonable_foodstuffs = list(reasonable_foodstuffs_set)
    
    reasonable_set = deshroomed_set # omitting the copy here (for negligible speed)
    reasonable_set -= salts_to_remove_set
    reasonable_set -= fat_oils_to_remove_set
    reasonable_set -= other_flavorings_to_remove_set
    reasonable_set -= foodstuffs_to_remove_set
    
    reasonable_list = list(reasonable_set)
    reasonable_length = len(reasonable_list)

    return (
        reasonable_n_salts,
        reasonable_n_fat_oils,
        reasonable_n_other_flavorings,
        reasonable_n_foodstuffs,
        reasonable_salts_set,
        reasonable_fat_oils_set,
        reasonable_other_flavorings_set,
        reasonable_foodstuffs_set,
        reasonable_salts,
        reasonable_fat_oils,
        reasonable_other_flavorings,
        reasonable_foodstuffs,
        reasonable_set,
        reasonable_list,
        reasonable_length
    )

reasonable_data = reasonable_clique_upper.apply(get_reasonable_data, axis=1)

CPU times: user 3.58 s, sys: 15.7 ms, total: 3.59 s
Wall time: 3.59 s


In [26]:
%%time

reasonable_clique_upper['reasonable_n_salts'] = reasonable_data.apply(lambda x: x[0])
reasonable_clique_upper['reasonable_n_fat_oils'] = reasonable_data.apply(lambda x: x[1])
reasonable_clique_upper['reasonable_n_other_flavorings'] = reasonable_data.apply(lambda x: x[2])
reasonable_clique_upper['reasonable_n_foodstuffs'] = reasonable_data.apply(lambda x: x[3])
reasonable_clique_upper['reasonable_salts_set'] = reasonable_data.apply(lambda x: x[4])
reasonable_clique_upper['reasonable_fat_oils_set'] = reasonable_data.apply(lambda x: x[5])
reasonable_clique_upper['reasonable_other_flavorings_set'] = reasonable_data.apply(lambda x: x[6])
reasonable_clique_upper['reasonable_foodstuffs_set'] = reasonable_data.apply(lambda x: x[7])
reasonable_clique_upper['reasonable_salts'] = reasonable_data.apply(lambda x: x[8])
reasonable_clique_upper['reasonable_fat_oils'] = reasonable_data.apply(lambda x: x[9])
reasonable_clique_upper['reasonable_other_flavorings'] = reasonable_data.apply(lambda x: x[10])
reasonable_clique_upper['reasonable_foodstuffs'] = reasonable_data.apply(lambda x: x[11])
reasonable_clique_upper['reasonable_set'] = reasonable_data.apply(lambda x: x[12])
reasonable_clique_upper['reasonable_list'] = reasonable_data.apply(lambda x: x[13])
reasonable_clique_upper['reasonable_length'] = reasonable_data.apply(lambda x: x[14])

CPU times: user 61.4 ms, sys: 0 ns, total: 61.4 ms
Wall time: 60.1 ms


In [27]:
# IMPORTANT don't remove:

reasonable_clique_upper = reasonable_clique_upper[reasonable_clique_upper['reasonable_list'].apply(lambda x: len(x) >= 2)]

In [28]:
%%time
  
G = nx.Graph()
def get_connections_data(clique_list):
    G.clear()
    G.add_nodes_from(clique_list)
    weighted_edges = []
    connections = []
    for i_1, name_1 in enumerate(clique_list[:-1]):
        for i_2, name_2 in enumerate(clique_list[i_1+1:], i_1+1):
            connection = stir_fry_flavor_data[name_1][name_2]
    
            # Really, since I'm using upper the only demerit values here will be .5333 and .4. but, I think I've adjusted variation to 0-1 anyway, so it should be ok (?)
            if connection[0] == 'c':
                pairs_with_demerit = .8
            elif connection[0] == 'd':
                pairs_with_demerit = .6666
            elif connection[0] == 'C':
                pairs_with_demerit = .5333
            elif connection[0] == 'D':
                pairs_with_demerit = .4
            else:
                print('OH NO! BAD PAIRING VALUE.')

            if connection[1] == '_':
                strength_demerit = .2
            elif connection[1] == 's':
                strength_demerit = .15
            elif connection[1] == 'S':
                strength_demerit = .1
            else:
                print('OH NO! BAD STRENGTH VALUE.')
                
            connection_length = pairs_with_demerit + strength_demerit
            weighted_edges.append((name_1, name_2, connection_length))
            connections.append((name_1, name_2, connection))
            
    G.add_weighted_edges_from(weighted_edges)
    path_length = nx.average_shortest_path_length(G, weight='weight')
    return (path_length, connections)

connections_data = reasonable_clique_upper['reasonable_list'].apply(get_connections_data)

CPU times: user 3.11 s, sys: 3.81 ms, total: 3.11 s
Wall time: 3.12 s


In [29]:
reasonable_clique_upper['path_length'] = connections_data.apply(lambda x: x[0])
reasonable_clique_upper['connections'] = connections_data.apply(lambda x: x[1])
reasonable_clique_upper['connection_values'] = connections_data.apply(lambda x: [c[2] for c in x[1]])

In [30]:
# !pip install networkx==2.4
# !pip install pyvis
# !pip install python-louvain
# !pip install matplotlib
import networkx as nx
from pyvis import network as net
import community
import matplotlib.cm as cm
import matplotlib.pyplot as plt

In [32]:
%%time

fruit_set = set(stir_fry_flavor_data[stir_fry_flavor_data['stir_fry_fruit'] == 'y']['name'])
protein_set = set(stir_fry_flavor_data[stir_fry_flavor_data['stir_fry_protein'] == 'y']['name'])
bean_set = set(stir_fry_flavor_data[stir_fry_flavor_data['stir_fry_protein_bean'] == 'y']['name'])
grain_set = set(stir_fry_flavor_data[stir_fry_flavor_data['stir_fry_grain'] == 'y']['name'])
    
max_length_upper = max(reasonable_clique_upper['reasonable_length'].tolist())

def get_score(row):
    
    # NODE LENGTH BONUS
    # scale lengths so that max is 10, then square to skew sampling toward long cliques, then scale to 1 max
    nodes_length_score = (row['reasonable_length'] * 10/max_length_upper)**2 / 100
    
    # FOOD GROUPS BONUS
    n_fruit = len(row['reasonable_set'].intersection(fruit_set))
    n_protein = len(row['reasonable_set'].intersection(protein_set))
    protein_score = (n_protein/2)**.5

    fruit_score = (n_fruit/2)**.5
    
    food_group_score = protein_score + fruit_score
    
    # PATH LENGTH BONUS
    path_length_score = 2.3/row['path_length'] - 3.1
    
    # RICE BEANS DEMERIT
    n_beans = len(row['reasonable_set'].intersection(bean_set))
    n_grains = len(row['reasonable_set'].intersection(grain_set))

    # trying to keep this low, to tack onto 0-1 score
    rice_beans_demerit = 0
    if n_beans == 2:
        rice_beans_demerit += .1
    elif n_beans > 2:
        rice_beans_demerit += .25
    if n_grains == 2:
        rice_beans_demerit += .1
    elif n_grains > 2:
        rice_beans_demerit += .25
        
    score = (nodes_length_score*1.6 + path_length_score*1 + food_group_score*.4)/3 - rice_beans_demerit
    if score > 0:
        return score
    else:
        return 0

reasonable_clique_upper['score'] = reasonable_clique_upper.apply(get_score, axis=1)
reasonable_clique_upper.sort_values('score', ascending=False, inplace=True) #IMPORTANT
reasonable_clique_upper.reset_index(drop=True, inplace=True)

CPU times: user 1.42 s, sys: 20 ms, total: 1.44 s
Wall time: 1.44 s


In [60]:
# reasonable_clique_upper.to_pickle(os.path.join(root_path, 'DATA/stir_fry_reasonable_clique_upper.pickle'))
# reasonable_clique_upper.to_pickle(os.path.join(root_path, '../data/stir_fry_reasonable_clique_upper.pickle'))
# reasonable_clique_upper = pd.read_pickle(os.path.join(root_path, 'DATA/stir_fry_reasonable_clique_upper.pickle'))

In [33]:
%%time

average_score_for_length = dict(reasonable_clique_upper.groupby('reasonable_length')['score'].mean())
def get_average_score(length):
    if length >= 2:
        return average_score_for_length[length]
    else:
        return 0

CPU times: user 3.57 ms, sys: 0 ns, total: 3.57 ms
Wall time: 2.92 ms


In [34]:
def get_weak_score(strong):
    if strong == 'Y':
        return 1
    elif strong == 'y':
        return 3
    else:
        return 9
                        
stir_fry_flavor_data['weak_score'] = stir_fry_flavor_data['strong'].apply(get_weak_score)

In [35]:
%%time

stir_fry_names = stir_fry_flavor_data['name']

stir_fry_g = nx.Graph()
stir_fry_g.add_nodes_from(stir_fry_names)
weighted_edges = []
for i_1, name_1 in enumerate(stir_fry_names[:-1]):
    for i_2, name_2 in enumerate(stir_fry_names[i_1+1:], i_1+1):
        connection = stir_fry_flavor_data[name_1][name_2]

        if connection[0] == 'c':
            pairs_with_demerit = .8
        elif connection[0] == 'd':
            pairs_with_demerit = .6666
        elif connection[0] == 'C':
            pairs_with_demerit = .5333
        elif connection[0] == 'D':
            pairs_with_demerit = .4

        if connection[1] == '_':
            strength_demerit = .2
        elif connection[1] == 's':
            strength_demerit = .15
        elif connection[1] == 'S':
            strength_demerit = .1

        connection_length = pairs_with_demerit + strength_demerit
        weighted_edges.append((name_1, name_2, connection_length))

stir_fry_g.add_weighted_edges_from(weighted_edges)

CPU times: user 382 ms, sys: 12 ms, total: 394 ms
Wall time: 393 ms


In [36]:
%%time
# 3.98s

stir_fry_shortest_path_lengths = {key: value for key, value in nx.shortest_path_length(stir_fry_g, weight='weight')}

CPU times: user 3.99 s, sys: 3.73 ms, total: 3.99 s
Wall time: 4 s


In [57]:
# stir_fry_shortest_path_lengths.to_pickle(os.path.join(root_path, 'DATA/stir_fry_shortest_path_lengths.pickle'))
# stir_fry_shortest_path_lengths = pd.read_pickle(os.path.join(root_path, 'DATA/stir_fry_shortest_path_lengths.pickle'))

AttributeError: 'dict' object has no attribute 'to_pickle'

In [61]:
# %%time

# import pickle
# with open(os.path.join(root_path, 'DATA/stir_fry_shortest_path_lengths.pickle'), 'wb') as handle:
#     pickle.dump(stir_fry_shortest_path_lengths, handle, protocol=pickle.HIGHEST_PROTOCOL)
# with open(os.path.join(root_path, '../data/stir_fry_shortest_path_lengths.pickle'), 'wb') as handle:
#     pickle.dump(stir_fry_shortest_path_lengths, handle, protocol=pickle.HIGHEST_PROTOCOL)
# with open(os.path.join(root_path, 'DATA/stir_fry_shortest_path_lengths.pickle'), 'rb') as handle:
#     stir_fry_shortest_path_lengths = pickle.load(handle)

### 7.2.2. Generator set-up
(stuff to do once/call)

In [37]:
%%time

n_present = len(stir_fry_flavor_data)
present = stir_fry_flavor_data.sample(n_present)
present_set = set(present['name'])
present_strong_set = set(present[present['strong'].isin(['Y', 'y'])]['name'])
not_present_set = set(stir_fry_flavor_data['name']) - present_set

# n_locked = random.randrange(0, 6)
n_locked = 3
locked = present.sample(n_locked) # random for now, but will be handed down
locked_set = set(locked['name'])
locked_fat_oils = locked[locked['stir_fry_fat_oil'] == 'y']
locked_salts = locked[locked['stir_fry_salt'] == 'y']
locked_other_flavorings = locked[(locked['stir_fry_flavoring'] == 'y') & (locked['stir_fry_salt'] != 'y')]
locked_foodstuffs = locked[(locked['stir_fry_fat_oil'] != 'y') & (locked['stir_fry_salt'] != 'y') & (locked['stir_fry_flavoring'] != 'y')] # why am I selecting not salt? already selecting not flavoring. don't wanna mess tho..
locked_fat_oils_set = set(locked_fat_oils['name'])
locked_salts_set = set(locked_salts['name'])
locked_other_flavorings_set = set(locked_other_flavorings['name'])
locked_foodstuffs_set = set(locked_foodstuffs['name'])

the_rest = present[~present['name'].isin(locked['name'])]
the_rest_set = set(the_rest['name'])
the_rest_fat_oils = the_rest[the_rest['stir_fry_fat_oil'] == 'y']
the_rest_salts = the_rest[the_rest['stir_fry_salt'] == 'y']
the_rest_other_flavorings = the_rest[(the_rest['stir_fry_flavoring'] == 'y') & (the_rest['stir_fry_salt'] != 'y')]
the_rest_foodstuffs = the_rest[(the_rest['stir_fry_fat_oil'] != 'y') & (the_rest['stir_fry_salt'] != 'y') & (the_rest['stir_fry_flavoring'] != 'y')]
the_rest_fat_oils_set = set(the_rest_fat_oils['name'])
the_rest_salts_set = set(the_rest_salts['name'])
the_rest_other_flavorings_set = set(the_rest_other_flavorings['name'])
the_rest_foodstuffs_set = set(the_rest_foodstuffs['name'])

n_additional_salts_needed = max(n_salts - len(locked_salts), 0)
n_additional_salts_actual = min(n_additional_salts_needed, len(the_rest_salts))
n_total_salts_actual = n_additional_salts_actual + len(locked_salts)

n_additional_fat_oils_needed = max(n_fat_oils - len(locked_fat_oils), 0)
n_additional_fat_oils_actual = min(n_additional_fat_oils_needed, len(the_rest_fat_oils))
n_total_fat_oils_actual = n_additional_fat_oils_actual + len(locked_fat_oils)

n_additional_other_flavorings_needed_min = max(n_other_flavorings_min - len(locked_other_flavorings), 0)
n_additional_other_flavorings_actual_min = min(n_additional_other_flavorings_needed_min, len(the_rest_other_flavorings))
n_total_other_flavorings_actual_min = n_additional_other_flavorings_actual_min + len(locked_other_flavorings)

n_additional_other_flavorings_needed_max = max(n_other_flavorings_max - len(locked_other_flavorings), 0) # yikes, I had this as min..
n_additional_other_flavorings_actual_max = min(n_additional_other_flavorings_needed_max, len(the_rest_other_flavorings))
n_total_other_flavorings_actual_max = n_additional_other_flavorings_actual_max + len(locked_other_flavorings)

n_additional_foodstuffs_needed_min = max(n_foodstuffs_min - len(locked_foodstuffs), 0)
n_additional_foodstuffs_actual_min = min(n_additional_foodstuffs_needed_min, len(the_rest_foodstuffs))
n_total_foodstuffs_actual_min = n_additional_foodstuffs_actual_min + len(locked_foodstuffs)

n_additional_foodstuffs_needed_max = max(n_foodstuffs_max - len(locked_foodstuffs), 0)
n_additional_foodstuffs_actual_max = min(n_additional_foodstuffs_needed_max, len(the_rest_foodstuffs))
n_total_foodstuffs_actual_max = n_additional_foodstuffs_actual_max + len(locked_foodstuffs)

CPU times: user 17.4 ms, sys: 24 µs, total: 17.4 ms
Wall time: 16 ms


In [38]:
%%time

ok_cliques = reasonable_clique_upper[:1000].copy()

def get_ok_data(row):
    updated_set = row['reasonable_set'] - not_present_set
    updated_salts_set = row['reasonable_salts_set'] - not_present_set
    updated_fat_oils_set = row['reasonable_fat_oils_set'] - not_present_set
    updated_other_flavorings_set = row['reasonable_other_flavorings_set'] - not_present_set
    updated_foodstuffs_set = row['reasonable_foodstuffs_set'] - not_present_set

    updated_n_salts = len(updated_salts_set)
    updated_n_fat_oils = len(updated_fat_oils_set)
    updated_n_other_flavorings = len(updated_other_flavorings_set)
    updated_n_foodstuffs = len(updated_foodstuffs_set)

    n_salts_so_far = len(updated_salts_set.union(locked_salts_set))
    n_salts_to_remove = max(n_salts_so_far - n_total_salts_actual, 0)
    salts_to_remove_from_set = updated_salts_set - locked_salts_set
    more_salts_to_remove_set = set(random.sample(salts_to_remove_from_set, n_salts_to_remove))

    n_fat_oils_so_far = len(updated_fat_oils_set.union(locked_fat_oils_set))
    n_fat_oils_to_remove = max(n_fat_oils_so_far - n_total_fat_oils_actual, 0)
    fat_oils_to_remove_from_set = updated_fat_oils_set - locked_fat_oils_set
    more_fat_oils_to_remove_set = set(random.sample(fat_oils_to_remove_from_set, n_fat_oils_to_remove))

    n_other_flavorings_so_far = len(updated_other_flavorings_set.union(locked_other_flavorings_set))
    n_other_flavorings_to_remove = max(n_other_flavorings_so_far - n_total_other_flavorings_actual_max, 0) # is this right?
    other_flavorings_to_remove_from_set = updated_other_flavorings_set - locked_other_flavorings_set
    more_other_flavorings_to_remove_set = set(random.sample(salts_to_remove_from_set, n_salts_to_remove))

    n_foodstuffs_so_far = len(updated_foodstuffs_set.union(locked_foodstuffs_set))
    n_foodstuffs_to_remove = max(n_foodstuffs_so_far - n_total_foodstuffs_actual_max, 0) # is this right? thinks so - it's keeping foodstuffs from above max
    foodstuffs_to_remove_from_set = updated_foodstuffs_set - locked_foodstuffs_set
    more_foodstuffs_to_remove_set = set(random.sample(foodstuffs_to_remove_from_set, n_foodstuffs_to_remove))

    ok_other_flavorings_set = updated_other_flavorings_set - more_other_flavorings_to_remove_set
    ok_foodstuffs_set = updated_foodstuffs_set - more_foodstuffs_to_remove_set

    ok_set = updated_set # not bothering w copy, for (negligible) speed reasons
    ok_set -= more_salts_to_remove_set
    ok_set -= more_fat_oils_to_remove_set
    ok_set -= more_other_flavorings_to_remove_set
    ok_set -= more_foodstuffs_to_remove_set

    ok_list = list(ok_set)
    ok_length = len(ok_list)
    ok_n_locked = len(ok_set.intersection(locked_set))
    ok_n_other_flavorings = len(ok_other_flavorings_set)
    ok_n_foodstuffs = len(ok_foodstuffs_set)

    old_score_bonus_factor = row['score']/get_average_score(row['reasonable_length']) # how above/below avg score was previously
    ok_score = get_average_score(ok_length)*old_score_bonus_factor
    ok_score_xtreme = ok_score**2 # skew scores way upward, for use as weights

    ok_strong_set = ok_set.intersection(present_strong_set)

    return (
        ok_set,
        ok_list,
        ok_length,
        ok_score,
        ok_score_xtreme,
        ok_n_locked,
        ok_strong_set,
        ok_n_other_flavorings,
        ok_n_foodstuffs,
    )
ok_data = ok_cliques.apply(get_ok_data, axis=1)

ok_cliques['ok_set'] = ok_data.apply(lambda x: x[0])
ok_cliques['ok_list'] = ok_data.apply(lambda x: x[1])
ok_cliques['ok_length'] = ok_data.apply(lambda x: x[2])
ok_cliques['ok_score'] = ok_data.apply(lambda x: x[3])
ok_cliques['ok_score_xtreme'] = ok_data.apply(lambda x: x[4])
ok_cliques['ok_n_locked'] = ok_data.apply(lambda x: x[5])
ok_cliques['ok_strong_set'] = ok_data.apply(lambda x: x[6])
ok_cliques['ok_n_other_flavorings'] = ok_data.apply(lambda x: x[7])
ok_cliques['ok_n_foodstuffs'] = ok_data.apply(lambda x: x[8])

ok_cliques = ok_cliques[ok_cliques['ok_list'].apply(lambda x: len(x) >= 2)]
ok_cliques = ok_cliques.sort_values('ok_score', ascending=False)


CPU times: user 297 ms, sys: 0 ns, total: 297 ms
Wall time: 298 ms


### 7.2.3. Generator
(contains code that will execute many times / call)

In [51]:
%%time

def get_sort_key(length_tuple):
    return length_tuple[1]

def n_possible_edges(n_nodes):
    return int((n_nodes*(n_nodes-1)) / 2)

def get_first_name_in_set(sorted_tuples, food_set):
    for sorted_tuple in sorted_tuples:
        if sorted_tuple[0] in food_set:
            return sorted_tuple[0]

top_score = None
n_iterations = 200
for iteration in range(n_iterations):
    n_total_other_flavorings_actual = random.randrange(n_total_other_flavorings_actual_min, n_total_other_flavorings_actual_max+1)
    n_total_foodstuffs_actual = random.randrange(n_total_foodstuffs_actual_min, n_total_foodstuffs_actual_max+1)

    n_additional_other_flavorings_actual = n_total_other_flavorings_actual - len(locked_other_flavorings)
    n_additional_foodstuffs_actual = n_total_foodstuffs_actual - len(locked_foodstuffs)

    final_cliques = ok_cliques[(ok_cliques['ok_n_other_flavorings'] <= n_additional_other_flavorings_actual) & (ok_cliques['ok_n_foodstuffs'] <= n_additional_foodstuffs_actual)]

    if len(final_cliques) > 0: # BLACK MAGIC
#         print('LET\'S DO MAGIC!')
        clique = final_cliques.sample(1, weights='ok_score_xtreme').iloc[0] # using xtreme to skew sample toward top

        try:
            clique_ingredients = present.loc[clique['ok_list']]
        except:
            print('MAJOR PROBLEM! Likely cause: clique_data contains not-present ingredients.')
            clique_ingredients = stir_fry_flavor_data.loc[clique['ok_list']]

        salts_so_far_set = clique['ok_set'].intersection(salt_set).union(locked_salts_set)
        fat_oils_so_far_set = clique['ok_set'].intersection(fat_oil_set).union(locked_fat_oils_set)
        other_flavorings_so_far_set = clique['ok_set'].intersection(other_flavoring_set).union(locked_other_flavorings_set)
        foodstuffs_so_far_set = clique['ok_set'].intersection(foodstuff_set).union(locked_foodstuffs_set)
        ingredients_so_far_set = clique['ok_set'].union(locked_set)

        n_additional_non_clique_salts = n_total_salts_actual - len(salts_so_far_set) # pretty sure there shouldn't be more salts so far than salts actual
        n_additional_non_clique_fat_oils = n_total_fat_oils_actual - len(fat_oils_so_far_set) # pretty sure there shouldn't be more salts so far than salts actual
        n_additional_non_clique_other_flavorings = n_total_other_flavorings_actual - len(other_flavorings_so_far_set)
        n_additional_non_clique_foodstuffs = n_total_foodstuffs_actual - len(foodstuffs_so_far_set)

        selected_ingredients = present[present['name'].isin(ingredients_so_far_set)]

        additional_salts_pool = the_rest_salts[~the_rest_salts['name'].isin(clique_ingredients['name'])]
        if n_additional_non_clique_salts > 0:
            additional_salts = additional_salts_pool.sample(n_additional_non_clique_salts, weights='weak_score')
            selected_ingredients = selected_ingredients.append(additional_salts)

        additional_fat_oils_pool = the_rest_fat_oils[~the_rest_fat_oils['name'].isin(clique_ingredients['name'])]
        if n_additional_non_clique_fat_oils > 0:
            additional_fat_oils = additional_fat_oils_pool.sample(n_additional_non_clique_fat_oils, weights='weak_score')
            selected_ingredients = selected_ingredients.append(additional_fat_oils)

        additional_other_flavorings_pool = the_rest_other_flavorings[~the_rest_other_flavorings['name'].isin(clique_ingredients['name'])]
        if n_additional_non_clique_other_flavorings > 0:
            additional_other_flavorings = additional_other_flavorings_pool.sample(n_additional_non_clique_other_flavorings, weights='weak_score')
            selected_ingredients = selected_ingredients.append(additional_other_flavorings)

        additional_foodstuffs_pool = the_rest_foodstuffs[~the_rest_foodstuffs['name'].isin(clique_ingredients['name'])]
        if n_additional_non_clique_foodstuffs > 0:
            additional_foodstuffs = additional_foodstuffs_pool.sample(n_additional_non_clique_foodstuffs, weights='weak_score')
            selected_ingredients = selected_ingredients.append(additional_foodstuffs)
    else: # REGULAR
#         print('REGULAR IT IS.')
        selected_ingredients = locked
        if n_additional_salts_actual > 0:
            selected_ingredients = selected_ingredients.append(the_rest_salts.sample(n_additional_salts_actual))
        if n_additional_fat_oils_actual > 0:
            selected_ingredients = selected_ingredients.append(the_rest_fat_oils.sample(n_additional_fat_oils_actual))
        if n_additional_other_flavorings_actual > 0:
            selected_ingredients = selected_ingredients.append(the_rest_other_flavorings.sample(n_additional_other_flavorings_actual))
        if n_additional_foodstuffs_actual > 0:
            selected_ingredients = selected_ingredients.append(the_rest_foodstuffs.sample(n_additional_foodstuffs_actual))

    selected_names = selected_ingredients['name'].values.tolist()

    selected_g = nx.Graph()
    selected_g.add_nodes_from(selected_names)

    for i_1, name_1 in enumerate(selected_names[:-1]):
        for i_2, name_2 in enumerate(selected_names[i_1+1:], i_1+1):
            connection = selected_ingredients[name_1][name_2]

            # Weights super guess-y
            if connection[0] == 'c':
                selected_g.add_edge(name_1, name_2, length=1, weight=.3) # prev .8
            elif connection[0] == 'd':
                # pairs_with_demerit = .5 # prev .6666
                selected_g.add_edge(name_1, name_2, length=.7, weight=.6) # prev .8
            elif connection[0] == 'C':
                # pairs_with_demerit = .4 # prev .5333
                selected_g.add_edge(name_1, name_2, length=.6, weight=.7) # prev .8
            elif connection[0] == 'D':
                # pairs_with_demerit = .3 # prev. .4
                selected_g.add_edge(name_1, name_2, length=.3, weight=1) # prev .8

    # Try again, friend
    if not nx.is_connected(selected_g):
#         print(str(iteration)+': NOT CONNECTED; SKIPPING TO NEXT ITERATION')
        continue

    score = 0

    # CONNECTED PAIRING BONUS ==============================================
    # I want this to be VERY important. I feel this holds a lot of the strength of recipe,
    # and also encompasses strength-ness and locked-ness
    # ranges from roughly (0 to 1) * 5, tho could be a lil over or under that range
    average_shortest_path_length = nx.average_shortest_path_length(selected_g, weight='length')
    # average_shortest_path_score = 1 / average_shortest_path_length * 1.2 - 1.1 # normalizes full house
    pairing_score = 1 / average_shortest_path_length * 1.5 - 1.3 # normalizes small pool (& doesn't do bad w full house)
    score += pairing_score * 5
    # print('AVERAGE SHORTEST PATH SCORE', average_shortest_path_score)

    # Used for both strength and locked bonus:
    node_degrees = list(selected_g.degree(weight='weight'))
    average_degree = sum([node_degree[1] for node_degree in node_degrees]) / len(node_degrees)

    # STRENGTH BONUS =======================================================
    strength_above_average = 0
    for node_degree in node_degrees:
        if selected_ingredients['strong'][node_degree[0]] == 'Y':
            strength_above_average += (node_degree[1] - average_degree)
        elif selected_ingredients['strong'][node_degree[0]] == 'y':
            strength_above_average += (node_degree[1] - average_degree) * .5
    strength_score = strength_above_average * .2 + .5 # reasonable for small and full pools
    # print('STRENGTH SCORE', strength_score)
    score += strength_score

    # LOCKED BONUS =========================================================
    locked_above_average = 0
    for node_degree in node_degrees:
        if node_degree[0] in locked_names:
            # print(node_degree[1])
            locked_above_average += node_degree[1] - average_degree
    locked_score = locked_above_average * .2 + .5 # close enough (has to cover few locked, lotta locked, small pool, big pool - yeesh.)
    # print('LOCKED SCORE', locked_score)
    score += locked_score * 2

    # FLAVOR BALANCE BONUS =================================================
    # ranges from roughly (0 to 1) * 1 (could be a lil over/under)
    n_sweet_lower = (selected_ingredients['sweet'] == 'y').sum()
    n_sweet_upper = (selected_ingredients['sweet'] == 'Y').sum()
    n_salty_lower = (selected_ingredients['salty'] == 'y').sum()
    n_salty_upper = (selected_ingredients['salty'] == 'Y').sum()
    n_sour_lower = (selected_ingredients['sour'] == 'y').sum()
    n_sour_upper = (selected_ingredients['sour'] == 'Y').sum()
    n_savory_lower = (selected_ingredients['savory'] == 'y').sum()
    n_savory_upper = (selected_ingredients['savory'] == 'Y').sum()
    n_bitter_lower = (selected_ingredients['bitter'] == 'y').sum()
    n_bitter_upper = (selected_ingredients['bitter'] == 'Y').sum()
    n_spicy_lower = (selected_ingredients['spicy'] == 'y').sum()
    n_spicy_upper = (selected_ingredients['spicy'] == 'Y').sum()

    sweet_score = min(n_sweet_lower/2 + n_sweet_upper, 1)
    salty_score = min(n_salty_lower/2 + n_salty_upper, 1)
    sour_score = min(n_sour_lower/2 + n_sour_upper, 1)
    savory_score = min(n_savory_lower/2 + n_savory_upper, 1)
    bitter_score = min(n_bitter_lower/2 + n_bitter_upper, 1)
    spicy_score = min(n_spicy_lower/2 + n_spicy_upper, 1)

    flavor_score = 0
    flavor_score += sweet_score*3 # rly want something sweet in there
    flavor_score += salty_score*.5 # can always use salt
    flavor_score += sour_score*2 # like me some sour
    flavor_score += savory_score*3 # LOVE me some savory
    flavor_score += bitter_score # idk
    flavor_score += spicy_score*2 # can be nice
    flavor_score = flavor_score * .2 - 1.3 # yields reasonable scores w full or small pool

    score += flavor_score
    # print('FLAVOR SCORE', flavor_score)

    # FOOD GROUPS BONUS ==========================================================================================
    if 'y' in selected_ingredients['stir_fry_protein'].values:
        protein_score = .5
    else:
        protein_score = 0

    if 'y' in selected_ingredients['stir_fry_fruit'].values:
        fruit_score = .5
    else:
        fruit_score = 0

    food_group_score = protein_score + fruit_score
    score += food_group_score
    # print('FOOD GROUP SCORE', food_group_score)

    # print()
    if top_score == None or score > top_score:
        top_selected_ingredients = selected_ingredients
        top_pairing_score = pairing_score
        top_strength_score = strength_score
        top_locked_score = locked_score
        top_flavor_score = flavor_score
        top_food_group_score = food_group_score
        top_score = score

clique_net = net.Network(notebook=True)

top_selected_names = top_selected_ingredients['name'].tolist()
nodes = top_selected_names

def get_color(row):
#     return 'grey'
    if row['stir_fry_veg'] == 'y':
        return 'green'
    elif row['stir_fry_fruit'] == 'y':
        return 'orange'
    elif row['stir_fry_protein'] == 'y':
        return 'brown'
    elif row['stir_fry_grain'] == 'y':
        return 'tan'
    else:
        return 'lightgrey'
nodes_color = top_selected_ingredients.apply(get_color, axis=1).tolist()

clique_net.add_nodes(
    nodes=nodes,
    color=nodes_color
)

for i_1, name_1 in enumerate(top_selected_names[:-1]):
    for i_2, name_2 in enumerate(top_selected_names[i_1+1:], i_1+1):
        connection = top_selected_ingredients[name_1][name_2]
#         if connection[0] == 'c':
#             clique_net.add_edge(name_1, name_2, physics=False, color='lightgrey')
        if connection[0] == 'd':
            clique_net.add_edge(name_1, name_2, physics=True, color='whitesmoke')
        elif connection[0] == 'C':
            clique_net.add_edge(name_1, name_2, physics=True, color='lightgrey')
        elif connection[0] == 'D':
            clique_net.add_edge(name_1, name_2, physics=True, color='black')

clique_net.show('clique_net.html')

CPU times: user 3.87 s, sys: 7.79 ms, total: 3.88 s
Wall time: 3.88 s
